In [2]:
import spacy
import urllib.request
from bs4 import BeautifulSoup
import re
import gensim

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Get the URL of the Wikipedia page you want to scrape
url = "https://en.wikipedia.org/wiki/Artificial_intelligence"

# Open the URL and read the content
html = urllib.request.urlopen(url).read()

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser').find_all('p')

# Extract the text from the page
text = ''.join([item.text for item in soup])

print(text)


Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software which enable machines to perceive their environment and uses learning and intelligence to take actions that maximize their chances of achieving defined goals.[1] Such machines may be called AIs.
AI technology is widely used throughout industry, government, and science. Some high-profile applications include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go).[2] However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications

In [3]:
def clean_text(text):
    text = re.sub(r'(\x1b\[([0-9;]+)m|\[\d+\])', '', text)
    text = re.sub(r'\n', ' ', text)
    return text


# Create a spaCy document from the text
doc = nlp(clean_text(text))

# Print the sentences in the document
for sentence in doc.sents:
  print(">>"+sentence.text)

>> Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems.
>>It is a field of research in computer science that develops and studies methods and software which enable machines to perceive their environment and uses learning and intelligence to take actions that maximize their chances of achieving defined goals.
>>Such machines may be called AIs.
>>AI technology is widely used throughout industry, government, and science.
>>Some high-profile applications include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go).
>>However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applic

In [4]:
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')
# Load intents from JSON file
with open('/content/intents.json', 'r') as file:
    intents = json.load(file)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
# Preprocess data
all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        # Tokenize each word in the pattern
        words = word_tokenize(pattern.lower())
        all_words.extend(words)
        xy.append((words, tag))

# Create vocabulary set
all_words = sorted(set(all_words))

In [6]:
# Create training data
X_train = []
y_train = []
for (pattern_words, tag) in xy:
    bag = [1 if word in pattern_words else 0 for word in all_words]
    X_train.append(bag)
    y_train.append(tags.index(tag))

X_train = np.array(X_train)
y_train = np.array(y_train)


In [7]:
# Define PyTorch dataset
class IntentDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.X_data = torch.tensor(X_train, dtype=torch.float32)
        self.y_data = torch.tensor(y_train, dtype=torch.long)

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

In [8]:
# Define neural network model
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:
# Initialize parameters
input_size = len(all_words)
hidden_size = 8
output_size = len(tags)
learning_rate = 0.001
num_epochs = 1000
batch_size = 8

# Create DataLoader
dataset = IntentDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

In [10]:
# Initialize model, loss, and optimizer
model = NeuralNet(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        outputs = model(words)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 2.4968
Epoch [200/1000], Loss: 0.4943
Epoch [300/1000], Loss: 0.6291
Epoch [400/1000], Loss: 0.0685
Epoch [500/1000], Loss: 0.0116
Epoch [600/1000], Loss: 0.0150
Epoch [700/1000], Loss: 0.0036
Epoch [800/1000], Loss: 0.0084
Epoch [900/1000], Loss: 0.0029
Epoch [1000/1000], Loss: 0.0021


In [12]:
torch.save(model.state_dict(), 'chatbot_model.pth')

In [13]:
# Function to predict intent
def predict_intent(sentence):
    model.load_state_dict(torch.load('chatbot_model.pth'))
    model.eval()
    with torch.no_grad():
        # Tokenize input sentence
        sentence_words = word_tokenize(sentence.lower())
        # Convert to bag of words
        X = [1 if word in sentence_words else 0 for word in all_words]
        # Convert to tensor
        X = torch.tensor(X, dtype=torch.float32)
        # Forward pass
        output = model(X)
        # Get predicted tag
        predicted_tag = tags[torch.argmax(output).item()]
        return predicted_tag

In [14]:
# Function to get response
def get_response(intent):
    for intent_data in intents['intents']:
        if intent_data['tag'] == intent:
            responses = intent_data['responses']
            return np.random.choice(responses)

# Chat
while True:
    user_input = input("You: ")
    intent = predict_intent(user_input)
    response = get_response(intent)
    print("Bot:", response)
    if intent == 'goodbye':
      break

You: Hello
Bot: Hi there!
You: what is your name
Bot: 
You: What's your name?
Bot: I'm a chatbot. You can call me WikiChat.
You: Tell me a joke
Bot: Why did the scarecrow win an award? Because he was outstanding in his field!
You: Thanks
Bot: You're welcome!
You: bye
Bot: Take care!
